In [ ]:
%cd ../

# Utils
import os, sys
import src.data
import src.model

## Set **seed**

In [ ]:
src.data.set_all_seeds()

## Data

### **Initial**

In [15]:
classes = []

In [16]:
texts = []
labels = []

### **Split**

In [17]:
from sklearn.model_selection import train_test_split

train_texts, valid_texts, train_labels, valid_labels = train_test_split(texts, labels, test_size=0.2, random_state=42) # stratify=labels

### Create **Datasets**

In [ ]:
dataset = src.data.TextClassificationDataset(texts, labels)

train_set = src.data.TextClassificationDataset(train_texts, train_labels)
valid_set = src.data.TextClassificationDataset(valid_texts, valid_labels)

### Create **DataLoader**

In [ ]:
from torch.utils.data import DataLoader

batch_size = 32
num_workers = 0 # ! Возникают ошибки

train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=num_workers, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, num_workers=num_workers, shuffle=False)

### ***Visualization***

In [ ]:
src.data.show_text_classification(dataset, classes=classes)

## **Models**

In [21]:
from torch import nn, optim
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [23]:
class CustomOutput(nn.Module):
    def __init__(self, model, output_transform=lambda out: out.logits):
        super().__init__()
        self.model = model
        self.output_transform = output_transform

    def forward(self, *args, **kwargs):
        return self.output_transform(self.model(*args, **kwargs))

    def __getattr__(self, name):
        if name in ('model', 'output_transform'):
            return super().__getattr__(name)
        return getattr(self.model, name)
    
    def __setattr__(self, name, value):
        if name in ('model', 'output_transform'):
            super().__setattr__(name, value)
        else:
            setattr(self.model, name, value)

### *Score*

In [24]:
scores = dict()

### **Model**: `bert-base-uncased`

In [ ]:
model_name = 'bert-base-uncased'

src.data.TextClassificationDataset.tokenizer = AutoTokenizer.from_pretrained(model_name)
src.data.TextClassificationDataset.max_length = 128

In [ ]:
model = CustomOutput(AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(classes), ignore_mismatched_sizes=True))
optimizer = optim.Adam(model.parameters(), lr=5e-5)

model_wrapped = src.model.Classifier(model, 'bert-base-uncased', optimizer)

In [ ]:
model_wrapped.fit(train_loader, valid_loader, 3)

In [28]:
scores[model_wrapped.best_score] = model_wrapped

### **Model**: `distilbert-base-uncased`

In [ ]:
model_name = 'distilbert-base-uncased'

src.data.TextClassificationDataset.tokenizer = AutoTokenizer.from_pretrained(model_name)
src.data.TextClassificationDataset.max_length = 128

In [ ]:
model = CustomOutput(AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(classes), ignore_mismatched_sizes=True))

optimizer = optim.Adam(model.parameters(), lr=5e-5)
model_wrapped = src.model.Classifier(model, 'distilbert-base-uncased', optimizer)

In [ ]:
model_wrapped.fit(train_loader, valid_loader, 3)

In [ ]:
scores[model_wrapped.best_score] = model_wrapped

## Result

In [ ]:
best_model_wrapped = scores[max(scores)]
best_model_wrapped.name

In [ ]:
import random

n = 5
for _ in range(n):
    idx = random.randrange(len(valid_set))
    batch = valid_set[idx]
    text = valid_set.get_item(idx)['text']
    prediction = best_model_wrapped.predict(batch)
    print(f"Text: {text[:80]}{'...' if len(text)>80 else ''}")
    print(f'Class: {classes[batch["labels"]]} - Predict: {classes[prediction]}')
    print()

## Submission

In [ ]:
test_texts = []

test_set = src.data.TextClassificationDataset(test_texts)

In [ ]:
predict_class_id = best_model_wrapped.predict(test_set)
predict_class_names = [classes[class_id] for class_id in predict_class_id]